# Download Reddit WritingPrompts Datasets

Download pre-existing Reddit story datasets from HuggingFace

- WritingPrompts: ~66k stories 
- Adult WritingPrompts: ~393k stories (absurdist)

## Part 1: Install Required Packages

In [12]:
# Install HuggingFace datasets library
!pip install datasets -q

## Part 2: Setup and Imports

In [1]:
# Add parent directory to path for config import
import sys
sys.path.append('..')

# Imports
import pandas as pd 
from datasets import load_dataset
from tqdm.notebook import tqdm

# Import config settings
from config import RAW_DATA_DIR, MIN_STORY_LENGTH, MAX_STORY_LENGTH

print("Imports successful!")
print(f"Output directory: {RAW_DATA_DIR}")
print(f"Story length filter: {MIN_STORY_LENGTH}-{MAX_STORY_LENGTH} words")

Imports successful!
Output directory: /Users/averylee/Desktop/Fiction Unlimited/notebooks/../data/raw
Story length filter: 10-50 words


## Part 3: Download All Datasets

Download all three Reddit story datasets from HuggingFace.

In [14]:
# Download curated dataset (~66k stories, SFW)
print("Downloading WritingPrompts Dataset")
print("Source: euclaise/WritingPrompts_curated")

wp_dataset = load_dataset("euclaise/WritingPrompts_curated")
print(f"Loaded {len(wp_dataset['train'])} stories from curated dataset\n")

Source: euclaise/WritingPrompts_curated
Loaded 66332 stories from curated dataset



In [ ]:
# # Download NSFW dataset
# print("Downloading Adult WritingPrompts Dataset (NSFW)")
# print("Source: nothingiisreal/Reddit-Dirty-And-WritingPrompts")

# adult_dataset = load_dataset("nothingiisreal/Reddit-Dirty-And-WritingPrompts")
# print(f"Loaded {len(adult_dataset['train'])} stories from NSFW dataset\n")

Source: nothingiisreal/Reddit-Dirty-And-WritingPrompts
Loaded 393200 stories from NSFW dataset



## Part 4: Explore the Datasets

Let's look at what we downloaded from each dataset

In [20]:
# Dataset structure
print(wp_dataset)

sample = wp_dataset['train'][0]
for key, value in sample.items():
    print(f"{key}: {value}")

DatasetDict({
    train: Dataset({
        features: ['body', 'comment_score', 'prompt', 'post_score'],
        num_rows: 66332
    })
})
body: Veronica stretched out lazily on her grandmother’s couch. Upstairs, the rest of the family were happily chatting away. Veronica sighed. She hated the insincerity of family events. Her parents called her discontent a ‘teenage phase’, but she hated how her family could only express their feelings in food. Her parents had been pushing sweet, rich food on her for months.

Tired of staring at the ceiling, she wandered over to Grandma’s cabinet filled with VCRs. 

“Just like them to have a basement full of crap.” Veronica muttered. “Would it kill them to buy a DVD player?”

Each tape was meticulously labeled with a name. It took Veronica a second to recognize many of the names – each tape was for a deceased family member. At the top of the pile, there was a tape marked ‘Veronica’. Maybe it was an old aunt?

Curious, she popped it into the VCR player.

In [ ]:
# # Adult dataset structure
# print(adult_dataset)

# sample = adult_dataset['train'][0]
# for key, value in sample.items():
#     print(f"{key}: {value}")

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'label', 'score'],
        num_rows: 393200
    })
})
prompt: The contract had made it sound like such easy money. Get locked in the portal-belt for a month or so, take a random dick or two a day, enjoy the occasional generous tongue, but ultimately get paid lazing around the house all day. You didn't expect to be one of the most popular 'Toys' on the app
completion: [{'role': 'user', 'content': "The contract had made it sound like such easy money. Get locked in the portal-belt for a month or so, take a random dick or two a day, enjoy the occasional generous tongue, but ultimately get paid lazing around the house all day. You didn't expect to be one of the most popular 'Toys' on the app"}, {'role': 'assistant', 'content': 'Portal Panties\n\nLacy looked at the app on her phone. All she had to do was hit the button marked "Accept" and her first shift would begin. She looked at the profile picture above the butt

## Part 5: Convert All Datasets to Our Format and Combine

Convert all datasets to our standard story format and combine them

In [ ]:
def convert_format(dataset, subreddit='', nsfw=False):
    stories = []

    for i, item in enumerate(tqdm(dataset['train'], desc=subreddit)):
        story_text = item.get('body', '') or item.get('story', '') or item.get('text', '') or item.get('response', '')
        
        # exclude empty stories 
        if not story_text:
            continue
        
        word_count = len(story_text.split())
        
        story = {
            "id": f"{subreddit}_{i}",
            "text": story_text,
            "title": item.get('prompt', '') or item.get('title', ''),
            "word_count": word_count,
            "source": 'r/' + subreddit,
            "nsfw": nsfw
        }
        
        stories.append(story)
    
    return stories 

In [ ]:
all_stories = []
all_stories.extend(convert_format(wp_dataset, all_stories, subreddit='WritingPrompts', nsfw=False))
# all_stories = convert_format(adult_dataset, all_stories, subreddit='DirtyWritingPrompts', nsfw=False)

WritingPrompts:   0%|          | 0/66332 [00:00<?, ?it/s]

In [45]:
print(f"Total stories: {len(all_stories)}")

Total stories: 66332


## Part 6: Save Combined Dataset

Save all stories to csv 

In [ ]:
output_path = RAW_DATA_DIR / "reddit_stories.csv" 

all_stories = pd.DataFrame(all_stories)
all_stories.head()

,id,text,title,word_count,subreddit,nsfw
0,WritingPrompts_0,Veronica stretched out lazily on her grandmoth...,What's on the tape?,199,WritingPrompts,False
1,WritingPrompts_1,I remember my first days as a doctor. I wore m...,Write a story about a hero gone bad,704,WritingPrompts,False
2,WritingPrompts_2,"Her toes are cracked slightly, worn from years...","Describe ""her"" or ""him"" (you know the one) wi...",365,WritingPrompts,False
3,WritingPrompts_3,The first thing you need is water. The second ...,How I Survived The Zombie Outbreak,193,WritingPrompts,False
4,WritingPrompts_4,"Oh white room, oh white room. You have ensnare...",You're trapped in a coma and live in a mental...,415,WritingPrompts,False


In [48]:
all_stories.to_csv(output_path, index=False)

In [2]:
import pandas as pd 
pd.set_option('display.max_colwidth', None)

df = pd.read_csv(RAW_DATA_DIR / "reddit_stories.csv")
df.head(10)

id  \
0  WritingPrompts_0   
1  WritingPrompts_1   
2  WritingPrompts_2   
3  WritingPrompts_3   
4  WritingPrompts_4   
5  WritingPrompts_5   
6  WritingPrompts_6   
7  WritingPrompts_7   
8  WritingPrompts_8   
9  WritingPrompts_9   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   